# Convex Combination Layer

In [ ]:
from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import Input
from keras import backend as K
import numpy as np

H = 40
T = 1024


class ConvexCombination(Layer):
    def __init__(self, **kwargs):
        super(ConvexCombination, self).__init__(**kwargs)

    def build(self, input_shape):
        batch_size, H, T = input_shape[0]
        self.l = self.add_weight(name='l0',
                                     shape=(3, 1),  # Adding one dimension for broadcasting
                                     initializer='uniform',  # Try also 'ones' and 'uniform'
                                     trainable=True)
        self.gamma =  self.add_weight(name='elmo_gamma',
                                     shape=(1, 1),  # Adding one dimension for broadcasting
                                     initializer='uniform',  # Try also 'ones' and 'uniform'
                                     trainable=True)
        super(ConvexCombination, self).build(input_shape)

    def call(self, x):
        # x is a list of two tensors with shape=(batch_size, H, T)
        h0, h1, h2 = x
        weights = K.softmax(self.l)
        return self.gamma * (weights[0] * h0 + weights[1] * h1 + weights[2] * h2)

    def compute_output_shape(self, input_shape):
        return input_shape[0]


h1 = Input(shape=(H, T))
h2 = Input(shape=(H, T))
h3 = Input(shape=(H, T))

cc = ConvexCombination()([h1, h2, h3])
model = Model(inputs=[h1, h2, h3],
              outputs=cc)